In [1]:
"""Libraries"""

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import time
import os
import random
from scipy.integrate import solve_ivp
from multiprocessing import Pool
import importlib

# dot.notation access to dictionary attributes
class dotdict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

# my own file:
import diffeq as de
importlib.reload(de)

<module 'diffeq' from 'C:\\Users\\p\\OneDrive - Budapesti Műszaki és Gazdaságtudományi Egyetem\\TDK\\diffeq.py'>

In [2]:
"""Create folder to save files"""

study_name = 'adaptive search try 2'
file_format = '.csv'
file_base_name = 'output_'
seperator = ','
line_end = '\n'

parent_dir = os.getcwd()
all_files = os.listdir(parent_dir)
exists = False
for file in all_files:
    if os.path.isdir(file):
        if os.path.basename(file) == study_name:
            exists = True
            
save_dir = os.path.join(parent_dir, study_name)
if not exists:
    os.mkdir(save_dir)
    
def create_line(data, seperator=',', line_end='\n'):
    line = ''
    for d in data:
        line += str(d) + seperator
    line = line[:-1]
    line += line_end
    return line
    
def new_file(save_dir, file_base_name, number=0, file_format='.txt'):
    file = os.path.join(save_dir, file_base_name + str(number) + file_format)
    f = open(file, 'w')
    first_line = create_line(de.datalist, seperator)
    f.write(first_line)
    return f

# Wrong solution

In [51]:
axis_names = ['R_E', 'ratio', 'P_inf', 'alfa_M', 'T_inf', 'surfactant', 'index']

grids = [
    dotdict(dict(    # 4000
        R_E=[0.1e-6, 30.0e-6, 20],
        ratio=[1.5, 30, 20],
        P_inf=[1.0e5, 100.0e5, 10],
        alfa_M=[0.2],
        T_inf=[278.15],
        surfactant=[1.0],
        index=[de.par_indexOfArgon],
    )),
    dotdict(dict(   # 9600
        R_E=[0.1e-6, 30.0e-6, 20],
        ratio=[1.5, 30, 20],
        P_inf=[1.0e5, 100.0e5, 10],
        alfa_M=[0.05, 0.35, 4],
        T_inf=[273.15+5.0, 273.15+50.0, 4],
        surfactant=[0.25, 1.0, 4],
        index=[de.par_indexOfArgon],
        num=150
    )),
    dotdict(dict(  # 10000
        R_E=[0.1e-6, 30.0e-6, 100],
        ratio=[1.5, 30, 100],
        P_inf=[1.0e5, 100.0e5, 10],
        alfa_M=[0.05, 0.35, 4],
        T_inf=[273.15+5.0, 273.15+50.0, 4],
        surfactant=[0.25, 1.0, 4],
        index=[de.par_indexOfArgon],
        num=100
    )),
    dotdict(dict(   # 500
        R_E=[0.1e-6, 30.0e-6, 100],
        ratio=[1.5, 30, 100],
        P_inf=[1.0e5, 100.0e5, 50],
        alfa_M=[0.05, 0.35, 4],
        T_inf=[273.15+5.0, 273.15+50.0, 4],
        surfactant=[0.25, 1.0, 4],
        index=[de.par_indexOfArgon],
        num=100
    )),
]

In [ ]:
search = []
number = 1
start = time.process_time()

for n, grid in enumerate(grids):
    print(f'\n{n}. run {(time.process_time() - start): .2f} s')
    ranges = dotdict(dict(    # stores possible values, an axis can take
        R_E=[], ratio=[], P_inf=[], alfa_M=[], T_inf=[], surfactant=[], index=[],
    ))
    points = dotdict(dict(
        points=[], #R_E=[], ratio=[], P_inf=[], alfa_M=[], T_inf=[], surfactant=[], index=[], Energy=[],
    ))
# fill ranges
    for axis in axis_names:
        div = grid[axis]
        if len(div) == 3:
            ranges[axis] = np.linspace(div[0], div[1], div[2])
            grid[axis].append(ranges[axis][1] - ranges[axis][0])    # step_size
        else:
            ranges[axis] = [div[0]]
            grid[axis].append(None)    # no step in this direction
            
# first run, bruteforce parameter sweep
    if n==0:
        ID = 0
        for R_E in ranges.R_E:
            for ratio in ranges.ratio:
                for P_inf in ranges.P_inf:
                    for alfa_M in ranges.alfa_M:
                        for T_inf in ranges.T_inf:
                            for surfactant in ranges.surfactant:
                                for index in ranges.index:
                                    if [ID, R_E, ratio, P_inf, alfa_M, T_inf, surfactant, index] not in points.points:
                                        points.points.append([ID, R_E, ratio, P_inf, alfa_M, T_inf, de.VapourPressure(T_inf), index, surfactant])
                                        ID += 1
# adaptive search around best                    
    else:
        ID = 0
        for point in search.points[:grid.num]:   # best num points from last run
            loc_range = dotdict(dict(
                R_E=[], ratio=[], P_inf=[], alfa_M=[], T_inf=[], surfactant=[], index=[],
            ))
            for i, axis in enumerate(axis_names):
                last_step_size = grids[n-1][axis][-1]
                if last_step_size == None:
                    loc_range[axis] = ranges[axis]
                else:
                    start = max(grid[axis][0], point[i+1] - last_step_size)
                    end = min(grid[axis][1], point[i+1] + last_step_size)
                    for value in ranges[axis]:
                        if start < value and value < end:
                            loc_range[axis].append(value)

            for R_E in loc_range.R_E:
                for ratio in loc_range.ratio:
                    for P_inf in loc_range.P_inf:
                        for alfa_M in loc_range.alfa_M:
                            for T_inf in loc_range.T_inf:
                                for surfactant in loc_range.surfactant:
                                    for index in loc_range.index:
                                        if [ID, R_E, ratio, P_inf, alfa_M, T_inf, surfactant, index] not in points.points:
                                            points.points.append([ID, R_E, ratio, P_inf, alfa_M, T_inf, de.VapourPressure(T_inf), index, surfactant])
                                            ID += 1                            

# calculate new points
    print(f'Start simulations {(time.process_time() - start): .2f} s')
    file = new_file(save_dir, file_base_name, number, file_format)
    number += 1
    with Pool() as pool:
        results = pool.imap_unordered(de.solve, points.points)

        for data in results:
            Energy = data[17]
            if Energy <= 0: Energy = 1e30
            points.points[data[0]].append(Energy)
            line = create_line(data, seperator, line_end)
            file.write(line)
            print(f'index: {data[0]}/{len(points.points)},   error: {data[1]},   steps: {data[2]},   runtime: {data[3]} s   |   R_E={data[4]*1e6} [um]; ' + 
                  f'ratio={data[5]}; P_inf={data[6]}; alfa_M={data[7]}; T_inf={data[8]} [K]; index={data[10]}; surfactant={data[11]};   Energy={data[17]: .0f} [MJ/kg]' +
                  '                                        ', end='\r')
    file.close()
# sort points by z 
    print(f'Finished simulations {(time.process_time() - start): .2f} s')
    points.points.sort(key=lambda point : point[-1])
    print(f'Finished sorting {(time.process_time() - start): .2f} s')
# save points
    search = points
    
end = time.process_time()
elapsed = end - start
print(f"\n{elapsed:.2f} s")


0. run  0.00 s
Start simulations  0.09 s


# Good solution

In [42]:
def calculate(to_eval, number = 0):
    file = new_file(save_dir, file_base_name, number, file_format)
    number += 1
    ID = 0
    ret = [point for point in to_eval]
    
    with Pool() as pool:
        results = pool.imap_unordered(de.solve, to_eval)

        for data in results:
            Energy = data[17]
            if Energy <= 0: Energy = 1e30
            ret[ID].append(Energy)
            data[0] = ID
            line = create_line(data, seperator, line_end)
            file.write(line)
            print(f'index: {ID}/{len(to_eval)},   error: {data[1]},   steps: {data[2]},   runtime: {data[3]} s   |   R_E={data[4]*1e6} [um]; ' + 
                  f'ratio={data[5]}; P_inf={data[6]}; alfa_M={data[7]}; T_inf={data[8]} [K]; index={data[10]}; surfactant={data[11]};   Energy={data[17]: .0f} [MJ/kg]' +
                  '                                        ', end='\r')
            ID += 1
            
    file.close()
    return ret

In [43]:
axis_names = ['R_E', 'ratio', 'P_inf', 'alfa_M', 'T_inf', 'surfactant', 'index']

grids = [
    dotdict(dict(    # 4000
        R_E = dotdict(dict(start=0.1e-6, end=30.0e-6, division=20)),
        ratio = dotdict(dict(start=1.5, end=30.0, division=20)),
        P_inf = dotdict(dict(start=1.0e5, end=100.0e5, division=10)),
        alfa_M = dotdict(dict(start=0.05, end=0.35, division=1)),
        T_inf = dotdict(dict(start=273.15+5.0, end=273.15+50.0, division=1)),
        surfactant = dotdict(dict(start=0.25, end=1.0, division=1)),
        index = dotdict(dict(start=de.par_indexOfArgon, end=de.par_indexOfArgon, division=1)),
    )),
    dotdict(dict(    # 10000
        R_E = dotdict(dict(start=0.1e-6, end=30.0e-6, division=200)),
        ratio = dotdict(dict(start=1.5, end=30.0, division=200)),
        P_inf = dotdict(dict(start=1.0e5, end=100.0e5, division=10)),
        alfa_M = dotdict(dict(start=0.05, end=0.35, division=1)),
        T_inf = dotdict(dict(start=273.15+5.0, end=273.15+50.0, division=1)),
        surfactant = dotdict(dict(start=0.25, end=1.0, division=1)),
        index = dotdict(dict(start=de.par_indexOfArgon, end=de.par_indexOfArgon, division=1)),
        num = 100
    )),
    dotdict(dict(    # 6400
        R_E = dotdict(dict(start=0.1e-6, end=30.0e-6, division=200)),
        ratio = dotdict(dict(start=1.5, end=30.0, division=200)),
        P_inf = dotdict(dict(start=1.0e5, end=100.0e5, division=10)),
        alfa_M = dotdict(dict(start=0.05, end=0.35, division=4)),
        T_inf = dotdict(dict(start=273.15+5.0, end=273.15+50.0, division=4)),
        surfactant = dotdict(dict(start=0.25, end=1.0, division=4)),
        index = dotdict(dict(start=de.par_indexOfArgon, end=de.par_indexOfArgon, division=1)),
        num = 100
    )),
    dotdict(dict(    # 18000
        R_E = dotdict(dict(start=0.1e-6, end=30.0e-6, division=200)),
        ratio = dotdict(dict(start=1.5, end=30.0, division=200)),
        P_inf = dotdict(dict(start=1.0e5, end=100.0e5, division=50)),
        alfa_M = dotdict(dict(start=0.05, end=0.35, division=10)),
        T_inf = dotdict(dict(start=273.15+5.0, end=273.15+50.0, division=10)),
        surfactant = dotdict(dict(start=0.25, end=1.0, division=4)),
        index = dotdict(dict(start=de.par_indexOfArgon, end=de.par_indexOfArgon, division=1)),
        num = 100
    )),
]

for i in range(len(grids)):
    for axis in axis_names:
        grids[i][axis].range = np.linspace(grids[i][axis].start, grids[i][axis].end, grids[i][axis].division)
        if grids[i][axis].division == 1:
            grids[i][axis].step = None
        else:
            grids[i][axis].step = grids[i][axis].range[1] - grids[i][axis].range[0]

In [ ]:
print('Run 0 grid operations')
points = [] 
for R_E in grids[0].R_E.range:
    for ratio in grids[0].ratio.range:
        for P_inf in grids[0].P_inf.range:
            for alfa_M in grids[0].alfa_M.range:
                for T_inf in grids[0].T_inf.range:
                    for surfactant in grids[0].surfactant.range:
                        for index in grids[0].index.range:
                            points.append([0, R_E, ratio, P_inf, alfa_M, T_inf, de.VapourPressure(T_inf), index, surfactant])

print('Run 0 start evals')
points = calculate(points, 0)
points.sort(key=lambda point : point[-1])
grids[0].points = points

for i in range(1, len(grids)):
    print(f'\nRun {i} grid operations')
    points = []
    for good_point in grids[i-1].points[:grids[i].num]:
        axis = 'R_E'; j = axis_names.index(axis)+1
        step = grids[i-1][axis].step
        R_E_range = [x for x in grids[i].R_E.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
        for R_E in R_E_range:
            axis = 'ratio'; j = axis_names.index(axis)+1
            step = grids[i-1][axis].step
            ratio_range = [x for x in grids[i].ratio.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
            for ratio in ratio_range:
                axis = 'P_inf'; j = axis_names.index(axis)+1
                step = grids[i-1][axis].step
                P_inf_range = [x for x in grids[i].P_inf.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
                for P_inf in P_inf_range:
                    axis = 'alfa_M'; j = axis_names.index(axis)+1
                    step = grids[i-1][axis].step
                    alfa_M_range = [x for x in grids[i].alfa_M.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
                    for alfa_M in alfa_M_range:
                        axis = 'T_inf'; j = axis_names.index(axis)+1
                        step = grids[i-1][axis].step
                        T_inf_range = [x for x in grids[i].T_inf.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
                        for T_inf in T_inf_range:
                            axis = 'surfactant'; j = axis_names.index(axis)+1
                            step = grids[i-1][axis].step
                            surfactant_range = [x for x in grids[i].surfactant.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
                            for surfactant in surfactant_range:
                                axis = 'index'; j = axis_names.index(axis)+1
                                step = grids[i-1][axis].step
                                index_range = [x for x in grids[i].index.range if good_point[j]-step < x and x < good_point[j]+step] if step != None else grids[i][axis].range
                                for index in index_range:
                                    point = [0, R_E, ratio, P_inf, alfa_M, T_inf, de.VapourPressure(T_inf), index, surfactant]
                                    if point not in points:
                                        points.append([0, R_E, ratio, P_inf, alfa_M, T_inf, de.VapourPressure(T_inf), index, surfactant])    
            
    print(f'Run {i} start evals')
    new_points = calculate(points, i)
    new_points.sort(key=lambda point : point[-1])
    grids[i].points = new_points

Run 0 grid operations
Run 0 start evals
index: 3999/4000,   error: 0,   steps: 172148,   runtime: 143.484375 s   |   R_E=30.0 [um]; ratio=30.0; P_inf=8900000.0; alfa_M=0.05; T_inf=278.15 [K]; index=9.0; surfactant=0.25;   Energy= 132549 [MJ/kg]                                                                                    
Run 1 grid operations
Run 1 start evals
